In [1]:
from pathlib import Path
import gzip
import pickle


def compression(path):
    """
    Based on filename, is compression being used?
    """
    compress = None
    ext = path.suffix
    if ext == ".gz":
        compress = True
    elif ext == ".pkl":
        compress = False
    else:
        raise Exception(f"invalid file extension [{ext}], must be .dil or .dil.gz")

    return compress

def write(info, data, fname, compressionlevel=3):
    """
    Stores data in a binary file using dill. This is better than pickle when
    using namedtuples to store data.
    info: dict holding sensor info/settings
    data: sensor data, [{},{},...,{}]
    fname: file name, file extension: *.dil=uncompressess, *.dil.gz=compressed
    compressionlevel: [low,fastest] 1-9 [highest,slowest]
    """
    bag = {
        "info": info,
        "data": data
    }

    path = Path(fname).expanduser()
    compress = compression(path)

    with path.open("wb") as fd:
        if compress:
            d = pickle.dumps(bag)
            d = gzip.compress(d, compresslevel=compressionlevel)
            fd.write(d)
        else:
            pickle.dump(bag, fd)

def read(fname):
    """
    Opens and reads a data file.
    fname: file name, file extension: *.dil=uncompressess, *.dil.gz=compressed
    """
    path = Path(fname).expanduser()
    compress = compression(path)

    with path.open("rb") as fd:
        if compress:
            d = fd.read()
            d = gzip.decompress(d)
            data = pickle.loads(d)
        else:
            data = pickle.load(fd)

    return data

In [2]:
from collections import namedtuple

Sensor = namedtuple("Sensor","x y z")

data = []
for i in range(10):
    data.append(Sensor(i,i,i))
    
write({"a": "hello"}, data, "test.pkl")
read("test.pkl")

In [5]:
write({"a": "hello"}, data, "test.pkl.gz")
read("test.pkl.gz")

{'info': {'a': 'hello'},
 'data': [Sensor(x=0, y=0, z=0),
  Sensor(x=1, y=1, z=1),
  Sensor(x=2, y=2, z=2),
  Sensor(x=3, y=3, z=3),
  Sensor(x=4, y=4, z=4),
  Sensor(x=5, y=5, z=5),
  Sensor(x=6, y=6, z=6),
  Sensor(x=7, y=7, z=7),
  Sensor(x=8, y=8, z=8),
  Sensor(x=9, y=9, z=9)]}